# Model 1 (trained on FER-2013)

In [72]:
import numpy as np
import pandas as pd
from tensorflow.python.lib.io import file_io
from skimage.transform import resize
from keras.models import load_model
import os
import cv2

import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import plotly.express as px

In [3]:
# Model used
train_model =  "ResNet"

In [4]:
# Size of the images
if train_model == "Inception":
    img_width, img_height =	139, 139
elif train_model == "ResNet":
    img_width, img_height =	197, 197

In [5]:
test_data_dir = "FER databases/FER-2013/fer2013/fer2013.csv"

In [6]:
model = load_model("FER models/ResNet-50.h5")

In [7]:
batch_size = 1

In [8]:
def preprocess_input(x):
    if train_model == "Inception":
        x /= 127.5
        x -= 1.
        return x
    elif train_model == "ResNet":
        x -= 128.8006  # np.mean(train_dataset)
        x /= 64.6497   # np.std(train_dataset)
    return x

In [9]:
def detect_faces(img):
    detected_faces = []
    # Load the cascade
    face_cascade = cv2.CascadeClassifier("face_detection_impl_file/haarcascade_frontalface_default.xml")
    # Convert into grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect faces
    faces = face_cascade.detectMultiScale(
        gray_img,
        scaleFactor		= 1.07,
        minNeighbors	= 5,
        # Ignore faces that are smaller than 30x30
        minSize			= (30, 30)) 

    for (x, y, w, h) in faces:
        # Extract the region of interest (face) from the image
        global raw_input_img
        ROI_gray = gray_img[y:y+h, x:x+w] 
        raw_input_img = img[y:y+h, x:x+w]
        detected_faces.append(ROI_gray)
    
    return detected_faces

In [ ]:
folder = '/Users/ruslankononov/Study/Programming/Python/CS50/fer/inputs'
for сategory in os.scandir(folder):
        if not сategory.name.startswith('.'):
            for image in os.scandir(сategory):
                face = detect_faces(image)[0]
                model_input = preprocess_input(face)
                prediction = model.predict(model_input)
                result = np.arg(prediction)

In [28]:
def evaluate_model(predictions, labels):
    predicted_classes   = np.argmax(predictions, axis = 1) 
    true_classes        =  labels                          
    class_names         = list(["Anger", "Disgust", "Fear", "Happinness", "Sadness", "Surprise", "Neutral"])

    accuracy = accuracy_score(
        true_classes, 
        predicted_classes, 
        normalize = True)

    report = classification_report(
        true_classes, 
        predicted_classes,
        target_names = class_names)

    print("Accuracy:")
    print(accuracy)
    print("\n")
    print("Report:")
    print(report)
    print("\n")

In [25]:
def get_data(dataset):
    file_stream = file_io.FileIO(test_data_dir, mode="r")
    data = pd.read_csv(file_stream)
    data = data[data['Usage']=='PrivateTest']
    pixels = data["pixels"].tolist()
    images = np.empty((len(data), img_height, img_width, 3))
    i = 0
    
    for pixel_sequence in pixels:
        single_image = [float(pixel) for pixel in pixel_sequence.split(" ")]	# Extraction of each image
        single_image = np.asarray(single_image).reshape(48, 48)					# Dimension: 48x48
        single_image = resize(single_image, (img_height, img_width), order = 3, mode = "constant") # Bicubic
        ret = np.empty((img_height, img_width, 3))  
        ret[:, :, 0] = single_image
        ret[:, :, 1] = single_image
        ret[:, :, 2] = single_image
        images[i, :, :, :] = ret
        i += 1
    
    images = preprocess_input(images)
    labels = data["emotion"].tolist()
    return images, labels

## Baseline accuracy

In [26]:
images, labels = get_data(test_data_dir)

In [27]:
%%time
predictions = model.predict(
    images,
    batch_size = batch_size)

CPU times: user 31min 45s, sys: 2min 2s, total: 33min 47s
Wall time: 9min 57s


In [29]:
evaluate_model(predictions, labels)

Accuracy:
0.7124547227640011


Report:
              precision    recall  f1-score   support

       Anger       0.64      0.64      0.64       491
     Disgust       0.73      0.65      0.69        55
        Fear       0.58      0.49      0.53       528
  Happinness       0.88      0.91      0.89       879
     Sadness       0.58      0.60      0.59       594
    Surprise       0.81      0.80      0.80       416
     Neutral       0.68      0.74      0.71       626

    accuracy                           0.71      3589
   macro avg       0.70      0.69      0.69      3589
weighted avg       0.71      0.71      0.71      3589





## Test model on CK+ dataset

In [52]:
indices = {0:list(),1:list(),2:list(),3:list(),4:list(),5:list()}
def load_ck_data(data_dir):
    images = np.empty((927, img_height, img_width, 3))
    labels = list()
    labels_names = {'surprise':5, 'fear':2, 'disgust':1, 'happy':3, 'sadness':4, 'anger':0, 'contempt':6}
    i = 0
    
    
    for сategory in os.scandir(data_dir):
        if not сategory.name.startswith('.'):
            for image in os.scandir(сategory):
                img = cv2.imread(image.path, 1)
                gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_resized = cv2.resize(gray_img, (img_width, img_height))
                ret = np.empty((img_width, img_height, 3)) 
                ret[:, :, 0] = img_resized
                ret[:, :, 1] = img_resized
                ret[:, :, 2] = img_resized
                images[i, :, :, :] = ret
                labels.append(labels_names[сategory.name])
                indices[labels_names[сategory.name]].append(i)
                i+=1
    images = preprocess_input(images)
    return images, labels 

In [53]:
images, labels = load_ck_data("FER databases/CK+48")

In [54]:
%%time
predictions = model.predict(
    images,
    batch_size = batch_size)

CPU times: user 8min 21s, sys: 35.5 s, total: 8min 56s
Wall time: 3min 12s


In [55]:
evaluate_model(predictions, labels)

Accuracy:
0.6634304207119741


Report:
              precision    recall  f1-score   support

       Anger       0.32      0.43      0.37       135
     Disgust       0.95      0.29      0.45       177
        Fear       0.59      0.29      0.39        75
  Happinness       0.92      1.00      0.96       207
     Sadness       0.42      0.61      0.50        84
    Surprise       0.97      0.90      0.94       249
     Neutral       0.00      0.00      0.00         0

    accuracy                           0.66       927
   macro avg       0.60      0.50      0.52       927
weighted avg       0.78      0.66      0.68       927





## Test model on JAFFE dataset

In [56]:
def load_jaffe_data(data_dir):
    images = np.empty((213, img_height, img_width, 3))
    labels = list()
    labels_names = {'SURPRISE':5, 'FEAR':2, 'DISGUST':1, 'HAPPY':3, 'SAD':4, 'ANGRY':0, 'NEUTRAL':6}
    i = 0
    
    
    for сategory in os.scandir(data_dir):
        if not сategory.name.startswith('.'):
            for image in os.scandir(сategory):
                img = cv2.imread(image.path, 1)
                gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_resized = cv2.resize(gray_img, (img_width, img_height))
                ret = np.empty((img_width, img_height, 3)) 
                ret[:, :, 0] = img_resized
                ret[:, :, 1] = img_resized
                ret[:, :, 2] = img_resized
                images[i, :, :, :] = ret
                labels.append(labels_names[сategory.name])
                i+=1
    images = preprocess_input(images)
    return images, labels 

In [57]:
images, labels = load_jaffe_data("FER databases/jaffe")

In [58]:
%%time
predictions = model.predict(
    images,
    batch_size = batch_size)

CPU times: user 1min 54s, sys: 7.2 s, total: 2min 1s
Wall time: 47.9 s


In [59]:
evaluate_model(predictions, labels)

Accuracy:
0.43661971830985913


Report:
              precision    recall  f1-score   support

       Anger       0.75      0.10      0.18        30
     Disgust       0.00      0.00      0.00        29
        Fear       0.42      0.47      0.44        32
  Happinness       0.83      0.77      0.80        31
     Sadness       0.23      0.61      0.33        31
    Surprise       0.90      0.60      0.72        30
     Neutral       0.35      0.47      0.40        30

    accuracy                           0.44       213
   macro avg       0.50      0.43      0.41       213
weighted avg       0.50      0.44      0.41       213





## Test model on RAF-DB dataset

In [60]:
emo_labels = pd.read_csv('FER databases/RAF data/list_patition_label.txt', 
                     header=None, names=['name','emotion'], delim_whitespace=True)

In [61]:
emo_labels['emotion'] -= 1

In [68]:
def load_raf_data(data_dir):
    images = np.empty((3068, img_height, img_width, 3))
    labels = list()
    i = 0
    
    for image in os.scandir(data_dir):
        img = cv2.imread(image.path, 1)
        img_name = image.name.replace('_aligned','')
        if 'test' in img_name:
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img_resized = cv2.resize(gray_img, (img_width, img_height))
            ret = np.empty((img_width, img_height, 3)) 
            ret[:, :, 0] = img_resized
            ret[:, :, 1] = img_resized
            ret[:, :, 2] = img_resized
            images[i, :, :, :] = ret
            
            labels.append(emo_labels[emo_labels.name==img_name]['emotion'])
            i+=1
            
            if (i%500 == 0):
                print('{} photos processed'.format(i))
       
    images = preprocess_input(images)
    return images, labels

In [69]:
images, labels = load_raf_data('FER databases/RAF data/aligned')

500 photos processed
1000 photos processed
1500 photos processed
2000 photos processed
2500 photos processed
3000 photos processed


In [70]:
%%time
predictions = model.predict(
    images,
    batch_size = batch_size)

CPU times: user 27min 27s, sys: 1min 54s, total: 29min 22s
Wall time: 9min 21s


In [71]:
evaluate_model(predictions, labels)

Accuracy:
0.560625814863103


Report:
              precision    recall  f1-score   support

       Anger       0.09      0.09      0.09       329
     Disgust       0.03      0.01      0.02        74
        Fear       0.06      0.07      0.07       160
  Happinness       0.92      0.81      0.86      1185
     Sadness       0.50      0.73      0.59       478
    Surprise       0.00      0.00      0.00       162
     Neutral       0.60      0.54      0.57       680

    accuracy                           0.56      3068
   macro avg       0.31      0.32      0.31      3068
weighted avg       0.58      0.56      0.56      3068





## Calculating Accuracy Drop

In [112]:
accs = [0.7125,0.6634,0.4366,0.5606]
accs_100 = [x*100 for x in accs]
datasets = ['FER-2013','CK+','JAFFE','RAF-DB']

fig = px.bar(x=datasets, y=accs_100, color=datasets, title="Accuracy by Dataset",
            labels = {'x':'Dataset', 'y':'Accuracy'}, text=accs_100)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.show()

In [114]:
base_acc = 71.25
accs_drop = [(1 - x / base_acc) * 100 for x in accs_100]
accs_drop.append(np.mean(accs_drop[1:]))
bar_names = ['CK+','JAFFE','RAF-DB','Average Drop']
colors = ['#EF553B','#00CC96','#AB63FA','#FFA15A'] 
fig = px.bar(x=bar_names, y=accs_drop[1:], title="Accuracy Drop by Dataset",
            labels = {'x':'Dataset', 'y':'Accuracy'}, text=accs_drop[1:])
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_traces(marker_color=colors)
fig.show()